In [56]:
# Import Dataset 
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Load the dataset
data = pd.read_excel("Wolf_Hormones.xlsx")
data.shape

(178, 7)

In [57]:
#Remove duplicate rows
data.drop_duplicates(inplace=True)
data.shape

(178, 7)

In [58]:
data.head(10)

,Individual,Sex,Population,Colour,Cpgmg,Tpgmg,Ppgmg
0,1,M,2,W,15.86,5.32,NaN
1,2,F,1,D,20.02,3.71,14.376220
2,3,F,2,W,9.95,5.30,21.659020
3,4,F,1,D,25.22,3.71,13.425070
4,5,M,2,D,21.13,5.34,NaN
5,6,M,2,W,12.48,4.60,NaN
6,7,M,1,W,26.78,4.58,NaN
7,8,M,1,D,15.41,9.27,NaN
8,9,F,1,D,33.87,4.81,19.912695
9,10,F,2,W,17.29,5.07,34.598060


In [59]:
#check the decoration
data.columns

Index(['Individual', 'Sex', 'Population', 'Colour', 'Cpgmg', 'Tpgmg', 'Ppgmg'], dtype='object')

## Data Exploration

In [60]:
# Data types of the columns
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Individual  178 non-null    int64  
 1   Sex         178 non-null    object 
 2   Population  178 non-null    int64  
 3   Colour      178 non-null    object 
 4   Cpgmg       178 non-null    float64
 5   Tpgmg       178 non-null    float64
 6   Ppgmg       69 non-null     float64
dtypes: float64(3), int64(2), object(2)
memory usage: 11.1+ KB


In [61]:
# Basic data statistics
data.describe()

,Individual,Population,Cpgmg,Tpgmg,Ppgmg
count,178.000000,178.000000,178.000000,178.000000,69.000000
mean,89.500000,1.915730,17.743258,6.147921,25.887460
std,51.528309,0.645436,8.921655,5.035901,8.921180
min,1.000000,1.000000,4.750000,3.140000,12.759095
25%,45.250000,1.250000,12.160000,4.372500,19.498915
50%,89.500000,2.000000,15.605000,5.070000,25.000370
75%,133.750000,2.000000,20.350000,6.317500,30.009250
max,178.000000,3.000000,73.190000,61.790000,53.281910


In [62]:
# Data statistics for object columns (Text columns)
data.describe(include=object)

,Sex,Colour
count,178,178
unique,3,3
top,M,W
freq,76,111


## Data Cleaning

In [63]:
# Check for missing values
data.isnull().sum()

Individual      0
Sex             0
Population      0
Colour          0
Cpgmg           0
Tpgmg           0
Ppgmg         109
dtype: int64

In [64]:
#Ppgmg has null values
# Replace missing values with the mean
data['Ppgmg'].fillna(data['Ppgmg'].mean(), inplace=True)

## Data Visualization